# 🏁 Sistema de Benchmark - YOLO

Este notebook demonstra o sistema completo de benchmark que testa automaticamente como diferentes parâmetros afetam o desempenho do modelo.

## Como Funciona

O sistema divide cada parâmetro em frações:
- 1/5: 20% do intervalo (min + 0.2 × range)
- 2/5: 40% do intervalo (min + 0.4 × range)
- 3/5: 60% do intervalo (min + 0.6 × range)
- 4/5: 80% do intervalo (min + 0.8 × range)
- 5/5: 100% do intervalo (valor máximo)

## Setup

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.core import Config, BenchmarkConfig, BenchmarkRunner

print("✅ Imports realizados com sucesso!")

## 1. Configuração Base

In [ ]:
# Configuração principal
config = Config(project_root=project_root)
config.model_name = 'yolo11n.pt'
config.device = 'cuda'
config.workers = 8

print(f"Projeto: {config.project_root}")
print(f"Resultados: {config.results_path}")

## 2. Dataset

**IMPORTANTE**: Atualize com o caminho do seu dataset!

In [ ]:
# ATUALIZE ESTE CAMINHO
dataset_path = '/path/to/your/data.yaml'

if not Path(dataset_path).exists():
    print(f"⚠️  Dataset não encontrado: {dataset_path}")
else:
    print(f"✅ Dataset: {dataset_path}")

## 3. Configuração do Benchmark

### Exemplo 1: Testar Epochs

In [ ]:
# Configuração do benchmark
benchmark_config = BenchmarkConfig()
benchmark_config.benchmark_name = 'epochs_benchmark'
benchmark_config.num_divisions = 5

# Define parâmetros para testar
benchmark_config.benchmark_params = {
    'epochs': {
        'min': 10,
        'max': 50,
        'type': 'int'
    }
}

# Parâmetros fixos
benchmark_config.fixed_params = {
    'batch_size': 16,
    'imgsz': 640
}

# Visualiza valores que serão testados
values = benchmark_config.get_benchmark_values('epochs')
print(f"Valores de epochs a testar: {values}")
print(f"Total de testes: {len(values)}")

### Exemplo 2: Múltiplos Parâmetros

In [ ]:
# Benchmark mais completo
benchmark_config_multi = BenchmarkConfig()
benchmark_config_multi.benchmark_name = 'multi_param_benchmark'
benchmark_config_multi.num_divisions = 5

benchmark_config_multi.benchmark_params = {
    'epochs': {'min': 10, 'max': 50, 'type': 'int'},
    'batch_size': {'min': 8, 'max': 32, 'type': 'int'},
    'lr0': {'min': 0.001, 'max': 0.1, 'type': 'float'},
    'optimizer': {'values': ['SGD', 'Adam', 'AdamW'], 'type': 'categorical'}
}

# Conta total de testes
total_tests = sum(
    len(benchmark_config_multi.get_benchmark_values(param))
    for param in benchmark_config_multi.benchmark_params
)

print(f"Total de testes a executar: {total_tests}")
print("\nParâmetros:")
for param, config in benchmark_config_multi.benchmark_params.items():
    values = benchmark_config_multi.get_benchmark_values(param)
    print(f"  {param}: {values}")

## 4. Executar Benchmark

### Modo Sequencial (Recomendado para começar)

In [ ]:
# ATENÇÃO: Este processo pode levar bastante tempo!
# Comece com poucos epochs para testar

# Cria runner
runner = BenchmarkRunner(config, benchmark_config)

# Executa (sequencial)
results = runner.run_benchmark(
    dataset_path=dataset_path,
    parallel=False
)

print("\n" + "="*60)
print("BENCHMARK CONCLUÍDO!")
print("="*60)
print(f"Total de testes: {results['total_tests']}")
print(f"Bem-sucedidos: {results['successful_tests']}")
print(f"Falhados: {results['failed_tests']}")
print(f"Tempo total: {results['total_time']/60:.2f} minutos")

### Modo Paralelo (Avançado)

**Atenção**: Requer múltiplas GPUs ou bastante RAM

In [ ]:
# Execução paralela (CUIDADO: pode consumir muitos recursos)
runner_parallel = BenchmarkRunner(config, benchmark_config)

results_parallel = runner_parallel.run_benchmark(
    dataset_path=dataset_path,
    parallel=True,
    max_workers=2  # Ajuste conforme recursos disponíveis
)

## 5. Análise Rápida dos Resultados

In [ ]:
import pandas as pd

# Extrai dados para DataFrame
data = []
for result in results['all_results']:
    if result.get('success', False):
        data.append({
            'param': result['benchmark_param'],
            'value': result['benchmark_value'],
            'training_time': result['training_time'] / 60,
            'mAP50-95': result.get('val_metrics', {}).get('mAP50-95', 0) * 100
        })

df = pd.DataFrame(data)
display(df)

# Estatísticas
print("\nEstatísticas por Parâmetro:")
print(df.groupby('param')['mAP50-95'].describe())

## 6. Visualização Básica

In [ ]:
import matplotlib.pyplot as plt

# Plot mAP vs parâmetro
for param in df['param'].unique():
    param_data = df[df['param'] == param]
    
    plt.figure(figsize=(10, 6))
    plt.plot(param_data['value'], param_data['mAP50-95'], marker='o', linewidth=2)
    plt.xlabel(param)
    plt.ylabel('mAP50-95 (%)')
    plt.title(f'Impacto de {param} no mAP')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Plot tempo
    plt.figure(figsize=(10, 6))
    plt.plot(param_data['value'], param_data['training_time'], marker='s', linewidth=2, color='orange')
    plt.xlabel(param)
    plt.ylabel('Tempo de Treinamento (min)')
    plt.title(f'Impacto de {param} no Tempo')
    plt.grid(True, alpha=0.3)
    plt.show()

## 7. Salvar Configuração do Benchmark

In [ ]:
# Salva configuração para reutilizar
config_file = config.results_path / 'benchmark_config.json'
benchmark_config.save(config_file)

print(f"✅ Configuração salva em: {config_file}")

## 8. Carregar Benchmark Anterior

In [ ]:
# Carrega resultados de benchmark anterior
benchmark_name = 'epochs_benchmark'  # Ajuste conforme necessário

try:
    loaded_results = runner.load_benchmark_results(benchmark_name)
    print(f"✅ Benchmark carregado: {benchmark_name}")
    print(f"Total de testes: {loaded_results['total_tests']}")
except FileNotFoundError:
    print(f"⚠️  Benchmark não encontrado: {benchmark_name}")

## Próximos Passos

Veja o notebook `04_analysis.ipynb` para:
- Visualizações interativas com Bokeh
- Análise estatística detalhada
- Rankings de impacto de parâmetros
- Relatórios automáticos